In [1]:
%%writefile fft_cuda.cu
#include <stdio.h>
#include <math.h>
#include <cuda_runtime.h>

// Define a simple structure for complex numbers
typedef struct {
    float x;  // Real part
    float y;  // Imaginary part
} Complex;

// CUDA kernel for FFT computation (Cooley-Tukey)
__global__ void fft_kernel(Complex *data, int N, int stage) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx >= N) return;

    int halfSize = 1 << stage;  // Size of the current sub-problem
    int step = 1 << (stage + 1); // Step size for the current stage

    // Calculate indices for the even and odd elements
    int evenIdx = idx;
    int oddIdx = idx + halfSize;

    if (oddIdx < N) {
        Complex even = data[evenIdx];
        Complex odd = data[oddIdx];

        // Twiddle factor (complex exponential)
        float angle = -2.0f * M_PI * (float)idx / (float)step;
        Complex twiddle = {cos(angle), sin(angle)};

        // Perform the FFT butterfly operation
        Complex temp;
        temp.x = twiddle.x * odd.x - twiddle.y * odd.y;
        temp.y = twiddle.x * odd.y + twiddle.y * odd.x;

        // Update the data array
        data[evenIdx].x = even.x + temp.x;
        data[evenIdx].y = even.y + temp.y;
        data[oddIdx].x = even.x - temp.x;
        data[oddIdx].y = even.y - temp.y;
    }
}

// Host code to initialize, run, and print results
void run_fft(Complex *data, int N) {
    Complex *d_data;
    cudaMalloc((void**)&d_data, sizeof(Complex) * N);
    cudaMemcpy(d_data, data, sizeof(Complex) * N, cudaMemcpyHostToDevice);

    // Perform FFT over multiple stages
    int threadsPerBlock = 256;
    int numBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;

    // Log2(N) stages
    int logN = log2f(N);
    for (int stage = 0; stage < logN; ++stage) {
        fft_kernel<<<numBlocks, threadsPerBlock>>>(d_data, N, stage);
        cudaDeviceSynchronize();
    }

    // Copy the result back to host
    cudaMemcpy(data, d_data, sizeof(Complex) * N, cudaMemcpyDeviceToHost);

    cudaFree(d_data);
}

int main() {
    int N = 16;  // FFT size (must be a power of 2)

    // Create input data (complex values, here using a simple sine wave)
    Complex *data = (Complex*)malloc(sizeof(Complex) * N);
    for (int i = 0; i < N; i++) {
        data[i].x = cos(2 * M_PI * i / N);  // Real part
        data[i].y = sin(2 * M_PI * i / N);  // Imaginary part
    }

    // Run FFT on the GPU
    run_fft(data, N);

    // Print the results
    for (int i = 0; i < N; i++) {
        printf("data[%d] = %f + %fi\n", i, data[i].x, data[i].y);
    }

    free(data);
    return 0;
}



Writing fft_cuda.cu


In [2]:
! nvcc fft_cuda.cu -o fft_cuda

In [3]:
! ./fft_cuda

data[0] = 4.568536 + 9.513482i
data[1] = 0.458804 + -2.521863i
data[2] = 3.082391 + -3.847759i
data[3] = 1.000000 + -0.414212i
data[4] = 6.568534 + -0.541195i
data[5] = -0.458804 + -0.306563i
data[6] = 0.000001 + 0.765366i
data[7] = 0.171571 + 0.414213i
data[8] = 6.568536 + -0.541196i
data[9] = -1.021412 + -0.306563i
data[10] = -1.910818 + -0.511402i
data[11] = -0.082392 + -0.198913i
data[12] = -3.486143 + -2.541197i
data[13] = 0.458804 + 0.306562i
data[14] = -0.000001 + 0.765368i
data[15] = 0.082393 + -0.034129i
